# Simultaneously fit multiple lightcurves with airmass detrending

Here we will fit multiple lightcurves simultaneously using the global fitter in exotic. We will set up bounds for each lightcurve that specify what parameters to fit along with 'global' bounds that are shared between all lightcurves. 

![](https://s3.amazonaws.com/aasie/images/0004-6256/164/5/null/ajac8deef3_hr.jpg)

This notebook will show you how to reproduce something similar to Figure 4 in a study on HD 80606 b, https://ui.adsabs.harvard.edu/abs/2022AJ....164..178P/abstract 

Please cite that paper if you use this code

## Find lightcurve data in the Exoplanet Watch Database

The command `get` will fetch priors, light curves and an ephemeris (O-C) for each target.

In [ ]:
import os
import numpy as np
from copy import deepcopy
from exotic.api.ew import ExoplanetWatch, translate_keys
from IPython.display import display, Image
import matplotlib.pyplot as plt
%matplotlib inline


# This will load the results JSON from the link above
EW = ExoplanetWatch()
print(EW.target_list)

# names are case and space sensitive
target ='TrES-4 b'
result = EW.get(target)

# directory to save outputs to
output_dir = target.replace(" ","_").replace("-","_")

# make the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# list the result properties
result.__dict__.keys()

## Priors
A list of stellar and planetary parameters with references. These are used to calculate a light curve model with non-linear limb darkening.

In [ ]:
result.priors

In [ ]:
# translate json to exotic compatible format
transit_priors = {
    'ecc': float(result.priors['ecc']['value']),
    'inc': float(result.priors['inc']['value']),
    'omega': float(result.priors['omega']['value']),
    'tmid': float(result.priors['Tc']['value']),
    'a1': 1, 'a2': 0, # airmass: a1 * exp(a2 * sec(z))
    'ars': float(result.priors['a/R*']['value']),
    'rprs': float(result.priors['Rp/R*']['value']),
    'per': float(result.priors['Period']['value']),
    # u0,u1,u2,u3 - non-linear limb darkening added later
}

## Observations
A list of observations for each target. These are used to fit the light curve model to the data.

In [ ]:
len(result.observations)

In [ ]:
# list the properties
print(result.observations[0].__dict__.keys())

# Download the light curve data + plot best fit

In [ ]:
# let's over plot the transit model too
from exotic.api.elca import transit

time, flux, fluxerr, airmass, airmasscorr = result.observations[0].get_data()

plt.plot(time, flux/airmasscorr, 'ko')
plt.plot(time, transit(time, result.observations[0].parameters), 'r-')
plt.xlabel("Time [BJD]")
plt.ylabel("Rel. Flux")

## Fit a single lightcurve using EXOTIC

In [ ]:
from exotic.api.elca import lc_fitter

# only define bounds for the "free" parameters during fitting
mybounds = {
    # [lower, upper]
    'rprs':[0,0.2],
    'tmid':[ result.ephemeris['Tc']-0.02,
             result.ephemeris['Tc']+0.02],
    'inc':[ result.observations[0].parameters['inc']-3, 
            min(90,result.observations[0].parameters['inc']+3)],
    'a2':[-0.5,0.5] # airmass curvature
}

# add limb darkening priors or generate them from a model (e.g. exotethys)
transit_priors['u0'] = result.observations[0].parameters['u0']
transit_priors['u1'] = result.observations[0].parameters['u1']
transit_priors['u2'] = result.observations[0].parameters['u2']
transit_priors['u3'] = result.observations[0].parameters['u3']

# fit the light curve
myfit = lc_fitter(time, flux, fluxerr, airmass, deepcopy(transit_priors), mybounds)

In [ ]:
myfit.plot_bestfit()
plt.show()

In [ ]:
# show the posterior distributions
myfit.plot_triangle()
plt.show()

# Fit multiple light curves

We're going to optimize the multi-lightcurve fit by first fitting the lightcurves individually to remove airmass variations and then we'll perform a joint simultaneous fit called "global" fit, which has shared parameters like Rp/Rs, Inclination, Tmid and Period across all the light curves.

In [ ]:
from astropy.time import Time
from copy import deepcopy
from exotic.api.elca import glc_fitter

# prep data for global fitter
input_data = []

# free parameters for each light curve
local_bounds = []

In [ ]:
# fit the individual light curves from Exoplanet Watch and detrend airmass extinction
for n,obs in enumerate(result.observations):

    try:
        time, flux, fluxerr, airmass, airmasscorr = obs.get_data()
    except:
        data = obs.get_data()
        time, flux, fluxerr, airmass = data[:,0], data[:,1], data[:,2], data[:,3]

    # add limb darkening priors or generate them from a model (e.g. exotethys)
    transit_priors['u0'] = result.observations[n].parameters['u0']
    transit_priors['u1'] = result.observations[n].parameters['u1']
    transit_priors['u2'] = result.observations[n].parameters['u2']
    transit_priors['u3'] = result.observations[n].parameters['u3']
    prior = deepcopy(transit_priors)

    # compute tmid closest to observations
    tmid = prior['tmid']
    per = prior['per']
    obs_time = 0.5*(time.min() + time.max())
    obs_tmid = tmid + per*np.round((obs_time - tmid)/per)

    # mask out nans and 0 errors
    mask = np.isnan(flux) | (fluxerr <= 0) | np.isnan(fluxerr) | (airmass <= 1)
    
    if mask.sum() == len(flux):
        continue

    time = time[~mask]
    flux = flux[~mask]
    fluxerr = fluxerr[~mask]
    airmass = airmass[~mask]

    # only define bounds for the "free" parameters during fitting
    mybounds = {
        # [lower, upper]
        'rprs':[0, prior['rprs']*2],
        'tmid':[ obs_tmid-0.05,
                 obs_tmid+0.05],
        'inc':[ prior['inc']-3, min(90,prior['inc']+3)],
        'a2':[-0.5,0.5] # airmass curvature
    }

    try:
        # fit the light curve
        myfit = lc_fitter(time, flux, fluxerr, airmass, prior, mybounds) 
    except:
        print(f"Failed to fit {obs.obscode['id']}")
        continue

    rprs2 = myfit.parameters['rprs']**2
    rprs2err = myfit.errors['rprs']*2*myfit.parameters['rprs']

    # residuals must be smaller than transit depth
    if myfit.res_stdev > prior['rprs']**2:
        print(f"Skipping {obs.obscode['id']} due to large residuals")
        continue
    elif rprs2-3*rprs2err < 0: # 3 sigma clip
        print(f"Skipping {obs.obscode['id']} due to null transit detection")
    elif np.std(flux) > 0.03:  # ignore noisy data
        print(f"Skipping {obs.obscode['id']} due to high scatter")
        continue
    else:
        prior = deepcopy(result.observations[n].parameters)

        # add data to list
        input_data.append({
            'time':time,
            'flux':myfit.detrended,
            'ferr':myfit.detrendederr,
            'airmass':np.zeros_like(time),
            'priors':prior,
            'name':f"{obs.obscode['id']}",

            # save individual fit parameters
            'parameters':myfit.parameters,
            'errors':myfit.errors,
            'res_std':myfit.res_stdev
        })

        # add parameter for individual airmass detrending
        local_bounds.append({}) 

        # print some timing info
        mint = Time(input_data[-1]['time'].min(),format='jd').isot
        maxt = Time(input_data[-1]['time'].max(),format='jd').isot
        maxphase = (input_data[-1]['time'].max() - prior['tmid'])/prior['per']
        minphase = (input_data[-1]['time'].min() - prior['tmid'])/prior['per']
        print(f"{input_data[-1]['name']} {len(input_data)} : {mint} ({minphase:.4f}) - {maxt} ({maxphase:.4f})")

## Run the next block of code only if you want to add TESS data to the global fit


In order to make lightcurves from TESS please inspect the file `tess.py` in the `examples/` directory. You will need to set up a new environment and then run the script based on instructions at the top of the file. Once you have the lightcurves, you can run the next block of code to add them to the global fit.

In [ ]:
import glob
from pylightcurve import exotethys

# generate limb darkening coefficients for TESS
get_prior = lambda key: float(result.priors[key]['value'])
u0,u1,u2,u3 = exotethys(get_prior('LOGG'), get_prior('T*'), get_prior('FE/H'), 'TESS', method='claret', stellar_model='phoenix')

# get files from local directory
tess_lightcurves = glob.glob(f"output/{target.replace(' ','_').replace('-','_')}/*_AAVSO.txt")
print(f"Found {len(tess_lightcurves)} TESS light curves")

# add TESS data
for n,lc in enumerate(tess_lightcurves):

    # read in data
    time, flux, fluxerr, airmass, airmasscorr = np.loadtxt(lc, unpack=True, delimiter=',')
    prior = deepcopy(transit_priors)
    prior['u0'] = u0
    prior['u1'] = u1
    prior['u2'] = u2
    prior['u3'] = u3
    prior['tmid'] = result.ephemeris['Tc']
    prior['per'] = result.ephemeris['Period']
    prior['a2'] = 0.0 # no airmass correction for TESS

    # compute tmid closest to observations
    tmid = prior['tmid']
    per = prior['per']
    obs_time = 0.5*(time.min() + time.max())
    obs_tmid = tmid + per*np.round((obs_time - tmid)/per)

    # only define bounds for the "free" parameters during fitting
    mybounds = {
        # [lower, upper]
        'rprs':[0, prior['rprs']*2],
        'tmid':[ obs_tmid-0.05,
                 obs_tmid+0.05],
        'inc':[ prior['inc']-5, min(90,prior['inc']+5)]
    }

    try:
        myfit = lc_fitter(time, flux, fluxerr, airmass, prior, mybounds) 
    except:
        print(f"Failed to fit {obs.obscode['id']}")
        continue
    
    # add data to list
    input_data.append({
        'time':time,
        'flux':flux,
        'ferr':fluxerr,
        'airmass':np.zeros(time.shape),
        'priors':prior,
        'name':f"TESS",

        # save individual fit parameters
        'parameters':myfit.parameters,
        'errors':myfit.errors,
        'res_std':myfit.res_stdev
    })

    # no airmass detrending
    local_bounds.append({})

    # print some timing info
    mint = Time(input_data[-1]['time'].min(),format='jd').isot
    maxt = Time(input_data[-1]['time'].max(),format='jd').isot
    maxphase = (input_data[-1]['time'].max() - prior['tmid'])/prior['per']
    minphase = (input_data[-1]['time'].min() - prior['tmid'])/prior['per']
    print(f"{input_data[-1]['name']} {len(input_data)} : {mint} ({minphase:.4f}) - {maxt} ({maxphase:.4f})")

In [ ]:
# bounds shared by all light curves
global_bounds = {
    'rprs':(prior['rprs']*0.75, prior['rprs']*1.33),
    'inc':(prior['inc']-5, min(90,prior['inc']+5)),
    'per':( result.ephemeris['Period']-0.001,
            result.ephemeris['Period']+0.001),
    'tmid':( result.ephemeris['Tc']-0.01,
             result.ephemeris['Tc']+0.01),
}

global_fit = glc_fitter(input_data, global_bounds, local_bounds, individual_fit=False, verbose=True)

In [ ]:
global_fit.plot_triangle()
plt.savefig(os.path.join(output_dir,"posteriors.png"),facecolor='white')

In [ ]:
global_fit.plot_bestfit(alpha=0.025, bin_dt = 15./60/24, title=f"{target} Global Fit",phase_limits='median', ylim_sigma=5)
plt.savefig(os.path.join(output_dir,"bestfit.png"),facecolor='white')

In [ ]:
global_fit.plot_bestfits()
plt.savefig(os.path.join(output_dir,"bestfits.png"),facecolor='white')

## LaTeX Output for Global Fit

In [ ]:
# create latex formatted table for global fit parameters
for key in global_fit.errors:
    print(f"{key} & {global_fit.parameters[key]:.5f} $\pm$ {global_fit.errors[key]:.7f} \\\\")

In [ ]:
# create column headers
header = "Name"
for key in input_data[0]['errors']:
    # skip airmass parameters
    if 'a1' in key or 'a2' in key:
        continue
    header += f" & {key}"
header += " & $\sigma_{res}$"
header += " \\\\"
print(header)

# create latex formatted string
for lcdata in input_data:
    fstring = lcdata['name']
    for key in lcdata['errors']:

        # skip airmass parameters
        if 'a1' in key or 'a2' in key:
            continue

        # add to string
        fstring += f" & {lcdata['parameters'][key]:.6f} $\pm$ {lcdata['errors'][key]:.6f}"

    # add res_std
    fstring += f" & {lcdata['res_std']:.6f}"
    fstring += f" \\\\"
    print(fstring)


## Mid-transit times and uncertainties from Nasa Exoplanet Archive

In [ ]:
nea_tmids = []
nea_tmids_err = []

for i in range(len(result.ephemeris['nea_tmids'])):
    clean_ref = result.ephemeris['nea_references'][i].replace('%20', ' ').replace('&amp;', '&')

    # remove ExoFOP-TESS values since we're using TESS data itself, no need to double count
    if 'ExoFOP-TESS' in clean_ref:
        continue

    nea_tmids.append(result.ephemeris['nea_tmids'][i])
    nea_tmids_err.append(result.ephemeris['nea_tmids_err'][i])

    print(f"{float(result.ephemeris['nea_tmids'][i]):.6f} +- {float(result.ephemeris['nea_tmids_err'][i]):.6f} : {clean_ref}")

## Fit an ephemeris to the mid-transit times

T_next = T_0 + n * Period

In [ ]:
from exotic.api.nested_linear_fitter import linear_fitter

# min and max values to search between for fitting the ephemeris
bounds = {
    'm':[ # orbital period
        global_fit.parameters['per']-10*global_fit.errors['per'], 
        global_fit.parameters['per']+10*global_fit.errors['per']
    ], 
    'b':[ # mid-transit time
        global_fit.parameters['tmid']-10*global_fit.errors['tmid'],
        global_fit.parameters['tmid']+10*global_fit.errors['tmid']
    ] 
}

# used to plot red overlay in O-C figure
prior = {
    'm':[global_fit.parameters['per'], global_fit.errors['per']],   # value from global
    'b':[global_fit.parameters['tmid'], global_fit.errors['tmid']]  # value from WLS
}

# extract data from individual light curves in global fit
tmids = [lcfit['parameters']['tmid'] for lcfit in input_data]
tmids_err = [lcfit['errors']['tmid'] for lcfit in input_data]
rprs2 = [lcfit['parameters']['rprs']**2 for lcfit in input_data]
rprs = [lcfit['parameters']['rprs'] for lcfit in input_data]
rprs_err = [lcfit['errors']['rprs'] for lcfit in input_data]
res_stdev = [lcfit['res_std'] for lcfit in input_data]
name = [lcfit['name'] for lcfit in input_data]
tmids_err = np.array(tmids_err)

# filter bad data
mask = (res_stdev < np.mean(rprs2)) & (np.mean(rprs) > 5 * np.mean(rprs_err)) & (tmids_err*24*60<5)
# TODO 3sigma clip away from global ephemeris

# apply mask
tmids = np.array(tmids)[mask]
tmids_err = np.array(tmids_err)[mask]

# combine the good data from reprocessing of Exoplanet Watch and NASA Exoplanet Archive
tmids = np.array(list(tmids) + nea_tmids, dtype=float)
tmids_err = np.array(list(tmids_err) + nea_tmids_err, dtype=float)

# fit the data and compare ephemeris to global light curve fit
lf = linear_fitter( tmids, tmids_err, bounds, prior=prior )

# plot the O-C diagram
fig,ax = lf.plot_oc(prior_name='Global Fit')
plt.tight_layout()
plt.savefig(os.path.join(output_dir,"oc_global.png"),facecolor='white')
plt.close()
Image(os.path.join(output_dir,"oc_global.png"))

## Search for periodic signals in the O-C data

In [ ]:
fig,ax = lf.plot_periodogram()
plt.tight_layout()
plt.savefig(os.path.join(output_dir,"periodogram.png"),facecolor='white')
plt.close()
Image(os.path.join(output_dir,"periodogram.png"))